In [ ]:
# 12-18-2022
# benjamin m hardy
# some key references:

# great intro reference is Noise in MRI - Al macovski
# https://pubmed.ncbi.nlm.nih.gov/8875425/
# another more recent interesting take:
# https://ieeexplore.ieee.org/document/5910383

# great description of Q-relationship:
# https://pubmed.ncbi.nlm.nih.gov/10680678/
# would be good to improve on this paper's assumptions and to show it experimentally.

# Overall great reference is Darasse and Ginefri
# https://pubmed.ncbi.nlm.nih.gov/14652180/

# key question: How does temperature, coil size, conductivity of sample, influence noise contributions
# figure 4 in Darasse and Ginefri is the experimental demonstration we would want to show.

